<h1>Jupyter Notebook do Trabalho de Computação Paralela e Distribuída</h1>
<h1>Computação Paralela Aplicada ao Problema de Sistemas de Recomendação</h1>
<h2>CEFET/RJ - PPCIC</h2>
<h3>Alunos:
    <ul>
      <li>Adalberto Mineiro de Andrade - adalberto.andrade@eic.cefet-rj.br</li>
      <li>Antonio Jose de Castro Filho - antonio.castro@eic.cefet-rj.br</li>
    </ul>  
</h3>
<h3>Professora: Rafaelli Coutinho - rafaelli.coutinho@cefet-rj.br</h3>
<h3>Orientador: Eduardo Ogasawara - eogasawara@cefet-rj.br</h3>

<h4 style="text-align: center;">Sem Paralelismo</h4>

<h6>Configuração do ambiente</h6>

In [1]:
import io
import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

<h6>Carga dos dados</h6>

In [2]:
## Dados dos Artistas (artist_data_reduzido.txt)
## cada linha possui dois valores ID e NOME do artista
artist = list()
with open('artist_data_reduzido.txt', encoding="utf-8") as openfileobject:
    for line in openfileobject:
        line = line[ :-1]  # retira o sinal de fim de linha '\n'
        str = line.split('\t') # separa as colunas da linha
        artist.append([int(str[0]), str[1]])  # cada linha com dois valores

print('Linha 1', artist[0])
print('Linha 2', artist[1])
print('Linha 3', artist[2])
print('Linha 4', artist[3])
print('...')
print('...')
print('...')
print('Total de linhas (artists): ', len(artist))

Linha 1 [1240105, 'André Visior']
Linha 2 [1240113, 'riow arai']
Linha 3 [1240132, 'Outkast & Rage Against the Machine']
Linha 4 [6776115, '小松正夫']
...
...
...
Total de linhas (artists):  30537


In [3]:
## Dados dos Apelidos (artist_alias_reduzido.txt)
## cada linha possui dois valores IDapelido e IDnome do artista
artistAlias = list()
with open('artist_alias_reduzido.txt', encoding="utf-8") as openfileobject:
    for line in openfileobject:
        line = line[ :-1]  # retira o sinal de fim de linha '\n'
        str = line.split('\t') # separa as colunas da linha
        artistAlias.append([int(str[0]), int(str[1])])  # cada linha com dois valores

print('Linha 1', artistAlias[0])
print('Linha 2', artistAlias[1])
print('Linha 3', artistAlias[2])
print('Linha 4', artistAlias[3])
print('...')
print('...')
print('...')
print('Total de linhas (artists alias): ', len(artistAlias))

Linha 1 [1027859, 1252408]
Linha 2 [1017615, 668]
Linha 3 [6745885, 1268522]
Linha 4 [1018110, 1018110]
...
...
...
Total de linhas (artists alias):  587


In [4]:
## Dados dos Usuários (user_artist_data_reduzido.txt)
## cada linha possui tres valores IDusuario, IDartista e um CONTADOR
userXartist = list()
with open('user_artist_data_reduzido.txt', encoding="utf-8") as openfileobject:
    for line in openfileobject:
        line = line[ :-1]  # retira o sinal de fim de linha '\n'
        str = line.split() # separa as colunas da linha
        userXartist.append([int(str[0]), int(str[1]), int(str[2])])  #Cada linha com tres valores

print('Linha 1', userXartist[0])
print('Linha 2', userXartist[1])
print('Linha 3', userXartist[2])
print('Linha 4', userXartist[3])
print('...')
print('...')
print('...')
print('Total de linhas (user X artists):', len(userXartist))

Linha 1 [1059637, 1000010, 238]
Linha 2 [1059637, 1000049, 1]
Linha 3 [1059637, 1000056, 1]
Linha 4 [1059637, 1000062, 11]
...
...
...
Total de linhas (user X artists): 49481


<h6>Transformando</h6>
<p>Criando arrays e DataFrames para ter mais flexibilidade.</p>

In [5]:
## artist
array_artist = np.asarray(artist)
df_artist = pd.DataFrame.from_records(artist, columns=('id', 'name'))
df_artist.info()
df_artist.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30537 entries, 0 to 30536
Data columns (total 2 columns):
id      30537 non-null int64
name    30537 non-null object
dtypes: int64(1), object(1)
memory usage: 477.2+ KB


,id,name
0,1240105,André Visior
1,1240113,riow arai
2,1240132,Outkast & Rage Against the Machine
3,6776115,小松正夫
4,1030848,Raver's Nature


In [6]:
## artistAlias
array_artistAlias = np.asarray(artistAlias)
df_artistAlias = pd.DataFrame.from_records(artistAlias, columns=('idAlias', 'idName'))
df_artistAlias.info()
df_artistAlias.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 2 columns):
idAlias    587 non-null int64
idName     587 non-null int64
dtypes: int64(2)
memory usage: 9.2 KB


,idAlias,idName
0,1027859,1252408
1,1017615,668
2,6745885,1268522
3,1018110,1018110
4,1014609,1014609


In [7]:
## userXartist
array_userXartist = np.asarray(userXartist)
df_userXartist = pd.DataFrame.from_records(userXartist, columns=('idUser', 'idArtist', 'counter'))
df_userXartist.info()
df_userXartist.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49481 entries, 0 to 49480
Data columns (total 3 columns):
idUser      49481 non-null int64
idArtist    49481 non-null int64
counter     49481 non-null int64
dtypes: int64(3)
memory usage: 1.1 MB


,idUser,idArtist,counter
0,1059637,1000010,238
1,1059637,1000049,1
2,1059637,1000056,1
3,1059637,1000062,11
4,1059637,1000094,1


<h6>Trocar apelidos por nomes originais</h6>

In [8]:
## Efetua as trocas na array de userXartist
for alias in artistAlias:
    indexes = np.where(array_userXartist[:, 1] == alias[0])[0]
    array_userXartist[indexes, 1] = alias[1]

## Transforma a array em lista novamente
userXartist = array_userXartist.tolist()
df_userXartist = pd.DataFrame.from_records(userXartist, columns=('idUser', 'idArtist', 'counter'))

<h6>Cria matriz esparsa de avaliações Usuários X Artistas</h6>

In [9]:
# Cria lista de usuarios, artistas e contador
users = list(np.sort(df_userXartist.idUser.unique()))
artists = list(np.sort(df_userXartist.idArtist.unique()))
plays = list(df_userXartist.counter)
 
# Linhas e colunas para os valores do contador (plays)
rows = df_userXartist.idUser.astype('category', categories = users).cat.codes
cols = df_userXartist.idArtist.astype('category', categories = artists).cat.codes

# Contrói uma matriz esparsa com users, items e número de execuções
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

# Exibe dados sobre a matriz
data_sparse

/home/antoniocastro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys
/home/antoniocastro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


<50x30100 sparse matrix of type '<class 'numpy.int64'>'
	with 48943 stored elements in Compressed Sparse Row format>

<p>Calculando o quão esparsa é a matriz:</p>

In [10]:
matrix_size = data_sparse.shape[0]*data_sparse.shape[1] # linhas x colunas = capacidade total
ocupied = len(data_sparse.nonzero()[0]) # itens realmente ocupados
sparsity = 100*(1 - (ocupied/matrix_size))

# Exibe a porcentagem de elementos não ocupados
sparsity

96.74797342192691

<h6>Função Alternating Least Squares (ALS)</h6>

In [11]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):

    # Calcula a confidence para cada valor
    confidence = sparse_data * alpha_val
    
    # Tamanho, número de usuários e de artistas
    user_size, artist_size = sparse_data.shape
    
    # Gera vetores X e Y e coloca valores aleatórios.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (artist_size, features)))
    
    #Calcula I e lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(artist_size)
    
    I = sparse.eye(features)
    lI = lambda_val * I
    
    # Para cada iteração computar X e Y
    print('Iterações a serem executadas: ', iterations)
    for i in range(iterations):
        print('Iteração em execução: ', i + 1)

        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Para todos os usuários
        for u in range(user_size):

            # Linha.
            u_row = confidence[u,:].toarray() 

            # Preferência.
            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0


            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Formula final.
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)


        # Para todos os artistas
        for i in range(artist_size):

            # Transpõe a coluna.
            i_row = confidence[:,i].T.toarray()

            # Preferência.
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Formula final.
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)
            
    print('Modelo treinado!!!')

    return X, Y    

<h6>Executando a função ALS - treinando o sistema de recomendação</h6>

In [12]:
user_vecs, item_vecs = implicit_als(data_sparse, iterations=5, features=20, alpha_val=40)

Iterações a serem executadas:  5
Iteração em execução:  1
Iteração em execução:  2
Iteração em execução:  3
Iteração em execução:  4
Iteração em execução:  5
Modelo treinado!!!


<h6>Achar itens similares a um dado item</h6>

In [13]:
# Permite encontrar o nome do artista atraves de seu ID
artist_lookup = df_artist[['id', 'name']].drop_duplicates() # Only get unique item/description pairs

# Artistas similares a Led Zeppelin, código: 1394.
item_id = 1394

print('Artistas semelhantes a:')
# Mostra o nome do Artista
print(artist_lookup.name.loc[artist_lookup.id == item_id].iloc[0])

# Qual a linha de Led Zeppelin?
item_vec = item_vecs[item_id].T

# Similaridade com outros (top 10).
scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
top_10 = np.argsort(scores)[::-1][:10]

artists = []
artist_scores = []

# Pega o nome e o score do artista
for idx in top_10:
    artists.append(artist_lookup.loc[idx].iloc[1])
    artist_scores.append(scores[idx])

similar = pd.DataFrame({'artist': artists, 'score': artist_scores})

print(similar)

Artistas semelhantes a:
Led Zeppelin
                                              artist      score
0                               Riccardo Eberspacher  80.879554
1  Michael Torke/John Harle (saxophone), Albany S...  79.542736
2                                       2raumwohnung  78.180950
3                                          Melodrome  77.733844
4                                         みうらじゅん&安斎肇  76.044759
5                                    Hey Big Spender  75.989954
6                                         Los Perros  75.109268
7                             Tom Jones & Pretenders  74.169892
8                     Ralphi Rosario Featuring Xavie  73.942883
9                                    Lomax All Stars  73.896904


<h6>Recomendações para um usuário</h6>

<p>Função de recomendações</p>

In [14]:
def recommend(user_id, data_sparse, user_vecs, item_vecs, artist_lookup, num_items=10):
    # Todos os artistas que o usuario ja ouviu
    user_interactions = data_sparse[user_id,:].toarray()
    
    # Elimina da recomendacao itens que o usuario ja ouviu.
    user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
    user_interactions[user_interactions > 1] = 0

    # Calculando as recomendacoes, produto de usuários . artistas
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    # Escalonar os valores
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions*rec_vector_scaled
   
    # Todos os indices dos artistas, o top "num_items". 
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    artists = []
    scores = []

    # Adiciona os nomes dos artistas
    for idx in item_idx:
        artists.append(artist_lookup.loc[idx].iloc[1])
        scores.append(recommend_vector[idx])

    # Cria dataframe com recomendações de artistas e sua pontuacao (0 à 1)
    recommendations = pd.DataFrame({'artist': artists, 'score': scores})
    
    return recommendations

<p>Identificação do usuário:</p>

In [15]:
# As recomendações serão feitas para o usuário ID 2064012
id_usuario = 2064012

<p>O que um usuário específico tem ouvido:</p>

In [16]:
# O que o usuário tem ouvido?
consumed_items = df_userXartist.loc[df_userXartist.idUser == id_usuario]

df1 = consumed_items
df1['id'] = df1.idArtist

df2 = df_artist
df2['id'] = df2.id

inner = pd.merge(df1, df2, on='id')

print(inner[['idArtist','name']])

    idArtist                        name
0    1000024                   Metallica
1    1000052               The Offspring
2    1000157                       AC/DC
3    1000183                   Disturbed
4    1000321            Bowling For Soup
5    1000329             Bloodhound Gang
6    1000427             New Found Glory
7    1000481                    Slipknot
8    1000570                Bad Religion
9    1001007             Third Eye Blind
10   1001597                      P.O.D.
11   1001597                      P.O.D.
12   1001779                Modest Mouse
13   1001909                    Interpol
14   1001943              Richard Cheese
15   1002122                         Rap
16   1002216                The Darkness
17   1002648            Men Without Hats
18   1003221                         CKY
19   1007286                 Dir en grey
20   1007993                 Denis Leary
21  10091879  www.amateurtransplants.com
22   1009455                     Group X
23   1010602    

/home/antoniocastro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


<p>Gerando recomendações para o usuário:</p>

In [17]:
posicao = users.index(id_usuario)
recommendations = recommend(posicao, data_sparse, user_vecs, item_vecs, artist_lookup)
print(recommendations)

                                   artist     score
0                               Joan Baez  1.000000
1                           Rachel Gaudry  0.913273
2                            DDR 7th Max2  0.846057
3                        Steve McClintock  0.819230
4  Barbara Manning - The Blood of Feeling  0.815366
5                          Carthy, Martin  0.811773
6                               Che-SHIZU  0.779293
7                               J Xaverre  0.757695
8                               The Ponys  0.713437
9                          George Dalaras  0.706673
